# The unreasonable effectiveness of Character-level Language Models
## (and why RNNs are still cool)

### [Yoav Goldberg](http://www.cs.biu.ac.il/~yogo)

RNNs, LSTMs and Deep Learning are all the rage, and a recent [blog post](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) by Andrej Karpathy is doing a great job explaining what these models are and how to train them.
It also provides some very impressive results of what they are capable of.  This is a great post, and if you are interested in natural language, machine learning or neural networks you should definitely read it. 

Go read it now, then come back here. 

You're back? good. Impressive stuff, huh? How could the network learn to immitate the input like that?
Indeed. I was quite impressed as well.

However, it feels to me that most readers of the post are impressed by the wrong reasons.
This is because they are not familiar with **unsmoothed maximum-liklihood character level language models** and their unreasonable effectiveness at generating rather convincing natural language outputs.

In what follows I will briefly describe these character-level maximum-likelihood langauge models, which are much less magical than RNNs and LSTMs, and show that they too can produce a rather convincing Shakespearean prose. I will also show about 30 lines of python code that take care of both training the model and generating the output. Compared to this baseline, the RNNs may seem somehwat less impressive. So why was I impressed? I will explain this too, below.

## Unsmoothed Maximum Likelihood Character Level Language Model 

The name is quite long, but the idea is very simple.  We want a model whose job is to guess the next character based on the previous $n$ letters. For example, having seen `ello`, the next characer is likely to be either a commma or space (if we assume is is the end of the word "hello"), or the letter `w` if we believe we are in the middle of the word "mellow". Humans are quite good at this, but of course seeing a larger history makes things easier (if we were to see 5 letters instead of 4, the choice between space and `w` would have been much easier).

We will call $n$, the number of letters we need to guess based on, the _order_ of the language model.

RNNs and LSTMs can potentially learn infinite-order language model (they guess the next character based on a "state" which supposedly encode all the previous history). We here will restrict ourselves to a fixed-order language model.

So, we are seeing $n$ letters, and need to guess the $n+1$th one. We are also given a large-ish amount of text (say, all of Shakespear works) that we can use. How would we go about solving this task?

Mathematically, we would like to learn a function $P(c | h)$. Here, $c$ is a character, $h$ is a $n$-letters history, and $P(c|h)$ stands for how likely is it to see $c$ after we've seen $h$.

Perhaps the simplest approach would be to just count and divide (a.k.a **maximum likelihood estimates**). We will count the number of times each letter $c'$ appeared after $h$, and divide by the total numbers of letters appearing after $h$. The **unsmoothed** part means that if we did not see a given letter following $h$, we will just give it a probability of zero.

And that's all there is to it.


### Training Code
Here is the code for training the model. `fname` is a file to read the characters from. `order` is the history size to consult. Note that we pad the data with leading `~` so that we also learn how to start.


In [1]:
from collections import *

def train_char_lm(fname, order=4):
    data = open(fname).read()
    lm = defaultdict(Counter)
    pad = "~" * order
    data = pad + data
    for i in range(len(data)-order):
        history, char = data[i:i+order], data[i+order]
        lm[history][char]+=1
    def normalize(counter):
        s = float(sum(counter.values()))
        return [(c,cnt/s) for c,cnt in counter.items()]
    outlm = {hist:normalize(chars) for hist, chars in lm.items()}
    return outlm

Let's train it on Andrej's Shakespears's text:

In [2]:
!wget http://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt

URL transformed to HTTPS due to an HSTS policy
--2018-09-25 15:51:38--  https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4573338 (4.4M) [text/plain]
Saving to: ‘shakespeare_input.txt.1’

shakespeare_input.t 100%[===================>]   4.36M  4.12MB/s    in 1.1s    

2018-09-25 15:51:39 (4.12 MB/s) - ‘shakespeare_input.txt.1’ saved [4573338/4573338]



In [2]:
lm = train_char_lm("shakespeare_input.txt", order=4)

Ok. Now let's do some queries:

In [3]:
lm['ello']

[('r', 0.059625212947189095),
 ('w', 0.817717206132879),
 ('u', 0.03747870528109029),
 (',', 0.027257240204429302),
 (' ', 0.013628620102214651),
 ('.', 0.0068143100511073255),
 ('?', 0.0068143100511073255),
 (':', 0.005110732538330494),
 ('n', 0.0017035775127768314),
 ("'", 0.017035775127768313),
 ('!', 0.0068143100511073255)]

In [4]:
lm['Firs']

[('t', 1.0)]

In [5]:
lm['rst ']

[('C', 0.09550561797752809),
 ('f', 0.011235955056179775),
 ('i', 0.016853932584269662),
 ('t', 0.05377207062600321),
 ('u', 0.0016051364365971107),
 ('S', 0.16292134831460675),
 ('h', 0.019261637239165328),
 ('s', 0.03290529695024077),
 ('R', 0.0008025682182985554),
 ('b', 0.024879614767255216),
 ('c', 0.012841091492776886),
 ('O', 0.018459069020866775),
 ('w', 0.024077046548956663),
 ('a', 0.02247191011235955),
 ('m', 0.02247191011235955),
 ('n', 0.020064205457463884),
 ('I', 0.009630818619582664),
 ('L', 0.10674157303370786),
 ('M', 0.0593900481540931),
 ('l', 0.01043338683788122),
 ('o', 0.030497592295345103),
 ('H', 0.0040128410914927765),
 ('d', 0.015248796147672551),
 ('W', 0.033707865168539325),
 ('K', 0.008025682182985553),
 ('q', 0.0016051364365971107),
 ('G', 0.0898876404494382),
 ('g', 0.011235955056179775),
 ('k', 0.0040128410914927765),
 ('e', 0.0032102728731942215),
 ('y', 0.002407704654895666),
 ('r', 0.0072231139646869984),
 ('p', 0.00882825040128411),
 ('A', 0.0056179

So `ello` is followed by either space, punctuation or `w` (or `r`, `u`, `n`), `Firs` is pretty much deterministic, and the word following `ist ` can start with pretty much every letter.

### Generating from the model
Generating is also very simple. To generate a letter, we will take the history, look at the last $order$ characteters, and then sample a random letter based on the corresponding distribution.

In [6]:
from random import random

def generate_letter(lm, history, order):
        history = history[-order:]
        dist = lm[history]
        x = random()
        for c,v in dist:
            x = x - v
            if x <= 0: return c

To generate a passage of $k$ characters, we just seed it with the initial history and run letter generation in a loop, updating the history at each turn.

In [7]:
def generate_text(lm, order, nletters=1000):
    history = "~" * order
    out = []
    for i in range(nletters):
        c = generate_letter(lm, history, order)
        history = history[-order:] + c
        out.append(c)
    return "".join(out)

### Generated Shakespeare from different order models

Let's try to generate text based on different language-model orders. Let's start with something silly:

### order 2:

In [9]:
lm = train_char_lm("shakespeare_input.txt", order=2)
print(generate_text(lm, 2))

Fir ame: all vice mesting any-he hild,
It se me an mand abat to ded,--tiome, any me, so the then a butinto Ale ithoutrineg wark you the thy
Will, I, be th hadys wilift! be eve cove is the and wour earromis of hare non. Gavert to and mucell, lestled tellood tood pood of ank faince could closeat twill, and deding Jund to by anot
Our hearther paysed boll be swer! I fre now, hisell ad hathe adefor: the thim.

Puche at told shorthours'tithe butairearbated
Thears, wines!

LEY:
MINCE:
In lame patucas ox-wargive ste dieve; ince--
Thin you:
Witimpat's art othestrumptiord, awarect here withis wery ast th th there frachanguis Joyarded, an mang.

Her put coll by oncill blerpride oth in are, a lin youl par:
Alt. He ou I he sund mage:
The of Cou ne.

Evel teds he knot nis felve youch JOHN:
Gooth, my Thorthe sot did we wity
Secondy, the nows ontle rusts and's by rithey? cake ace on the I be witneyet And
As fall tioul my wousy liegrome me of eye they;
Wit hought,
Alanst tiongs to reron, the hes to wic

Not so great.. but what if we increase the order to 4?

### order 4

In [10]:
lm = train_char_lm("shakespeare_input.txt", order=4)
print(generate_text(lm, 4))

First Pomfrevives us: 'tis as freel of death by this calamation, my particular with three; we wish;
And I, sir.

LUCIUS:
It grant Menterprison'd: so I have him one envenomouse.

IACHIO:
My noble never of please.

POINS:
Who didst thout thus away?
But him
To busines wood, now than is that me speak no catch'd fort
Tickly me again us wrath,
Which noble gate,
Here subject your guard sleep
No, bind here,
Wantiquid do but suddenly
consequenet!

MALCOLM:
Who, I should out,
Which, yester thank men! cannot opposes
Submissiez votard, thou are--as back and on think'd the more sister this before our father of a press'd bring.

LORENZO:
It speech you.
For I robb'd up to benefit
A drunkards; and new-boys! Why, her.
And directly of such--
What out.

DEMETRIUS:
Is the now
One patience
Free a bring-woment.
Rome?
I have born.

ALBANY:
When reeking;
That, who mine is you perform'd, pomps we ape, sail have reven of his hast so comfort, gentle forthly partial when fall for doubt is in gorge spoke will and 

In [11]:
lm = train_char_lm("shakespeare_input.txt", order=4)
print(generate_text(lm, 4))

First Love.
One thy habit the kin, it's return to petition as villaint.

PERICLES:
Ay, if Cumber of and cloud, our again.
See, Duke of God be poor bears? What can any use him I found.

First-born
No tell marriven with the with may all it did, i' the cold
my spoke wine enemy,
I would, have wine, but by and myself. But from my sound
As, for a fearful thee.

QUINCE EDWARD POLONIUS:
By heart!

HIPPOLYTA:
Is it be betwixt title of our right,
When possible,
Kin the
dost pass?
Or else Silvia?
An odds;
She inferench! Commend:
I would not for you have doth gazed
Reven it like eart Merling,
Who with love this land horse,
Take simples I may prince,--
A speak,
Because to-morrow a committer,
Not me not.

OSWALD:
Make at hath noble Titus' journ two atter;
For heart!

BELARIUS:
Ambition, wit was negligent poor shall thee, coward such adventomb:
That I disable my old mother Juliet!
Our did her rascals, them.
Trebonius, the man heads heave I do; and in the kind
The is too long I hearth tress:
Good by e

This is already quite reasonable, and reads like English. Just 4 letters history! What if we increase it to 7?

### order 7

In [11]:
lm = train_char_lm("shakespeare_input.txt", order=7)
print(generate_text(lm, 7))

First Carrier:
Ay; is't not prove our general is condition,
And say thy health, to prey. Now, heaven, what man shifts of new-found oaths thou resolved rather look'd
And turn he came the prince and to win?

CRESSIDA:
It was the air to stand oppress,
'Tis well as I, may change your mother rejoice in me, thou pluck a bard sentences, must his island.

IACHIMO:
By Jupiter is the world, Isbel
the word, sir!' is very absent her?

JULIA:
Be it lawful king,
And prize in this plighter this, and rest to-morrow, bids you not tempt him to single man
to your table-book, ballad-mongers;
Which, thence,
And this son;
for my coat, I have drunkard's child.

ISABELLA:
Why does infect.
Ajax commanded here, take your hour.
I will make you may do you not be
matched, unless a parlous boy? henceforth gallant?

HAMLET:

LADY CAPULET:
Marry, farewell.
Thus to rail
on one sore that ruin happen.
What showers: we must my extremity; and this revenged on man,
And talks of men,
In that stock, Sir Topas, Sir Thomas Mow

### How about 10?

In [12]:
lm = train_char_lm("shakespeare_input.txt", order=10)
print(generate_text(lm, 10))

First Citizen:
Why answer not the worst.
For thee, oppression and of learning likewise have we to lose,
This mould of form,
The observed:
He hath not a tomb so evident
That you stand good father; cry 'Saint George,
Inspire me, that holp'st to kill them up
In their different greeting well; his cares are now in actions blacker than the sheets! That I might prick
The goer-back. Why came you between us,
That comes in my father
And like an angry look;
He thinks, being then most humbly do desire to know of you,
Of whence are you?
Your name, sir.

PETRUCHIO:
Here, stand behind me!
If thou hadst been toss'd from him.

EMILIA:
You told a lie, an odious, damned Dane,
Drink off this grounds of feed
Are now on sale, and at our more consider'd, let us not break with her.
Pray you, let me not be put out of question,
Two other sons, who in the wars.

MORTIMER:
These promises no element
In such a ring as this,
When time is spent, our pilgrimage;
Thy word is current when it was ourself in person.

CYMB

### This works pretty well

With an order of 4, we already get quite reasonable results. Increasing the order to 7 (~word and a half of history) or 10 (~two short words of history) already gets us quite passable Shakepearan text. I'd say it is on par with the examples in Andrej's post. And how simple and un-mystical the model is!

### So why am I impressed with the RNNs after all?

Generating English a character at a time -- not so impressive in my view. The RNN needs to learn the previous $n$ letters, for a rather small $n$, and that's it. 

However, the code-generation example is very impressive. Why? because of the context awareness. Note that in all of the posted examples, the code is well indented, the braces and brackets are correctly nested, and even the comments start and end correctly. This is not something that can be achieved by simply looking at the previous $n$ letters. 

If the examples are not cherry-picked, and the output is generally that nice, then the LSTM did learn something not trivial at all.

Just for the fun of it, let's see what our simple language model does with the linux-kernel code:

In [14]:
!wget http://cs.stanford.edu/people/karpathy/char-rnn/linux_input.txt

URL transformed to HTTPS due to an HSTS policy
--2018-09-25 15:52:32--  https://cs.stanford.edu/people/karpathy/char-rnn/linux_input.txt
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6206996 (5.9M) [text/plain]
Saving to: ‘linux_input.txt.1’

linux_input.txt.1   100%[===================>]   5.92M  4.25MB/s    in 1.4s    

2018-09-25 15:52:34 (4.25 MB/s) - ‘linux_input.txt.1’ saved [6206996/6206996]



In [16]:
lm = train_char_lm("linux_input.txt", order=10)
print(generate_text(lm, 10))


 *     wait->flags, root->kf_root = NULL;
#endif /* CONFIG_DEBUG_STACK_USAGE
static void unfreeze_cgroup(freezer);
		}
	} else
		memset(remcom_out_buffer, -EINVAL);
	}
}

/**
 * task_clear_jobctl_pending(current))
		timeout = MAX_SCHEDULE_TIMEOUT;

int __ref profile_init();
	power_kobj = kobject_init(work, &work_debug_hint,
	.fixup_activate - clears head page with the other meaning of PTR_TO_MAP_VALUE_OR_NULL
		 */
		if (css->ss->cancel_attach,
	.cancel_attach)
			css->ss->attach(css_tg(css);

	return false;
}

#else /* !GENERIC_MSI_DOMAIN_OPS */

static const char **argv)
{
	long n = *pos - 1;
			p = s_next(m, file, &l);
		if (!avail)
			goto out_unlock;

		if (rt_mutex_unlock(&bdev->bd_mutex);
	return 0;
	n = ctx->major) &&
			      opt != PR_SET_MM_END_DATA:
		mm->start_data))
			goto out_put;

	pwq->nr_active) ||
		    __put_user(r.rlim_cur = 0x7FFFFFFF;
	if (x.rlim_max = RLIM_INFINITY)
			r.rlim_cur > new_rlim->rlim_cur = RLIM_INFINITY) {
		retval = copy_thread(clone_flags & CLON

In [17]:
lm = train_char_lm("linux_input.txt", order=15)
print(generate_text(lm, 15))

~~~~
 *     wait->flags &= ~WQ_FLAG_WOKEN;		condition = true;
 *     smp_mb() // B				smp_wmb(); // C
 *						wait->flags |= WQ_FLAG_WOKEN;

	return default_wake_function(wait_queue_t *wait, int state)
{
	unsigned int futex_shift;
	unsigned int pdu)
{
	struct blk_trace *bt = q->blk_trace;

	if (likely(!irqd_is_setaffinity_pending(data);
		irq_copy_pending(struct irq_desc *desc)
{
	struct irq_desc *desc)
{
	irq_state_set_disabled(desc);
	desc->depth = 1;
	if (desc->irq_data.domain;

		if (domain) {
			struct irq_data *data)
{
	for (data = data->parent_data)
			irq_domain_free_irqs - Free IRQ number and associated lock. */
static void FETCH_FUNC_NAME(method, string) == fn) ||	\
	  (FETCH_FUNC_NAME(bitfield, type)(struct pt_regs *regs, void *dest)
{
	long ret;
	int maxlen  = get_rloc_len(*dl);
			/* Trick here, convert data_rloc to data_loc */
			*dl = convert_rloc_to_loc(*dl,
				 ent_size + tp->args[i].offset);
	}
}

extern void start_bandwidth_timer(struct hrtimer *period_timer, ktime_

In [18]:
lm = train_char_lm("linux_input.txt", order=20)
print(generate_text(lm, 20))

/*
 * linux/kernel/irq/msi.c
 *
 * Copyright (C) 1992 Darren Senn
 */

/* These are all the functions necessary to implement itimers */

#include <linux/string.h>
#include <linux/ftrace.h>

#include "trace.h"

static DEFINE_PER_CPU(struct sched_domain *sd,
				 struct sched_domain *sd, struct sched_group *sg;
	int i = task_cpu(p);

	if (curr_cpu == target_cpu)
		return 0;

	env->imbalance = DIV_ROUND_CLOSEST(
		sds->busiest_stat.avg_load * sds->busiest_stat.group_capacity,
		SCHED_CAPACITY_SCALE;

	/* Move if we gain throughput */
	if (capa_move > capa_now)
		env->imbalance = busiest->load_per_task =
			min(busiest->load_per_task;
		return;
	}

	/*
	 * OK, we don't have enough imbalance to justify moving tasks,
	 * however we may be able to increase total CPU capacity used by
	 * moving them.
	 */

	capa_now += busiest->group_capacity <
	    busiest->load_per_task, busiest->avg_load);
	capa_now /= SCHED_CAPACITY_SCALE * sds.total_load)
						/ sds.total_capacity;

	/*
	 * If the busies

In [19]:
print(generate_text(lm, 20))

/*
 * linux/kernel/itimer.c
 *
 * Copyright (C) 2008, 2009 Steffen Klassert <steffen.klassert@secunet.com>
 *
 * This program is free software; you can redistribute it and/or modify it
 * under the terms and conditions of the GNU General Public License as published by
 * the Free Software Foundation; either version
 * 2 of the Licence, or (at your option) any later version.
 *
 * This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
 * MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU
 * General Public License for more details.
 *
 * You should have received a copy of the GNU General Public
 * License as published by
 * the Free Software Foundation; either version 2 of the License, or
 * (at your option) any later version.
 *
 *  This program is free software; you can redistribute it and/or
 * modify it under the terms of the GNU General Public License
 *  as published by the Free Software Found

In [20]:
print(generate_text(lm, 20, nletters=5000))

/*
 * linux/kernel/irq/manage.c
 *
 * Copyright (C) 2000 Stephane Eranian <eranian@hpl.hp.com>
 * Xscale (R) modifications copyright (C) 2003 Intel Corporation.
 * Copyright (C) 2008, 2009 Steffen Klassert <steffen.klassert@secunet.com>
 *
 * This program is free software; you can redistribute it and/or modify
 * it under the terms of the GNU General Public License (GPL).
 *
 * This file contains functions which manage clock event device to the minimum delay.
 * @dev:	device to program
 *
 * Returns 0 on success and < 0 on failure.
 */
int srcu_notifier_chain_unregister(&module_notify_list, nb);
}
EXPORT_SYMBOL(unregister_console);

int unregister_console(struct console *co, const char *s,
   unsigned count)
{
	rq->nr_running -= count;
}

static int current_css_set_cg_links_read(struct seq_file *m, void *v)
{
	struct cgroup *parent, *cgrp;
	struct cgroup_subsys *ss;
	unsigned long key;
	int i;

	/*
	 * Build the set of subsystem state objects generated in
	 * find_existing_css_set(stru

Order 10 is pretty much junk. In order 15 things sort-of make sense, but we jump abruptly between the 
and by order 20 we are doing quite nicely -- but are far from keeping good indentation and brackets. 

How could we? we do not have the memory, and these things are not modeled at all. While we could quite easily enrich our model to support also keeping track of brackets and indentation (by adding information such as "have I seen ( but not )" to the conditioning history), this requires extra work, non-trivial human reasoning, and will make the model significantly more complex. 

The LSTM, on the other hand, seemed to have just learn it on its own. And that's impressive.

## The End

In [21]:
from IPython.core.display import HTML

def css_styling():
    styles = open("../css/notebook.css", "r").read()
    return HTML(styles)
css_styling()